In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd  # Import Pandas

base_url = "https://www.iplt20.com/teams/{team_name}/squad/{season}"


teams = [
    "chennai-super-kings", "delhi-capitals", "gujarat-titans", "kolkata-knight-riders",
    "lucknow-super-giants", "mumbai-indians", "punjab-kings", "rajasthan-royals",
    "royal-challengers-bangalore", "sunrisers-hyderabad"
]


season_ranges = {
    "chennai-super-kings": [2008, 2024],
    "delhi-capitals": [2008, 2024],
    "gujarat-titans": [2022, 2024],
    "kolkata-knight-riders": [2008, 2024],
    "lucknow-super-giants": [2022, 2024],
    "mumbai-indians": [2008, 2024],
    "punjab-kings": [2008, 2024],
    "rajasthan-royals": [2008, 2024],
    "royal-challengers-bangalore": [2008, 2024],
    "sunrisers-hyderabad": [2008, 2024]
}


player_details_list = []


def fetch_player_details(player_url):
    print(f"Fetching player details from: {player_url}")  
    player_response = requests.get(player_url)
    if player_response.status_code != 200:
        print(f"Failed to retrieve player details from {player_url}")
        return None  

    player_soup = BeautifulSoup(player_response.content, 'html.parser')

    player_details = {}


    img_div = player_soup.find('div', class_='membr-details-img position-relative')
    if img_div:
        img_tag = img_div.find('img')
        if img_tag and 'src' in img_tag.attrs:
            player_details['Image URL'] = img_tag['src'] 


    name_div = player_soup.find('div', class_='plyr-name-nationality')
    if name_div:
        h1_tag = name_div.find('h1')
        if h1_tag:
            player_details['Name'] = h1_tag.text.strip()  
    

    player_details['Player URL'] = player_url  


    return player_details if player_details else None

def scrape_team_season(team, season):
    url = base_url.format(team_name=team, season=season)
    print(f"Scraping team: {team}, season: {season} from URL: {url}")  
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to retrieve {url}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')
    ul_lists = soup.find_all('ul', class_='mb-0')

    for ul in ul_lists:
        anchors = ul.find_all('a')
        for a in anchors:
            player_link = a['href']
            
            player_details = fetch_player_details(player_link)
            if player_details:
                player_details_list.append(player_details)  

                player_name = player_details.get('Name', 'Unknown Player')
                print(f"Added player: {player_name}")  

for team in teams:
    start_year, end_year = season_ranges[team]
    for season in range(start_year, end_year + 1):
        if team in ['chennai-super-kings', 'rajasthan-royals'] and season in [2015, 2016]:
            print(f"Skipping {team} for season {season}")  
            continue
        scrape_team_season(team, str(season))

player_df = pd.DataFrame(player_details_list)

player_df.to_csv('ipl_player_details.csv', index=False)
print("Player details scraped and saved to 'ipl_player_details.csv'.")


Scraping team: chennai-super-kings, season: 2008 from URL: https://www.iplt20.com/teams/chennai-super-kings/squad/2008
Fetching player details from: https://www.iplt20.com/teams/chennai-super-kings/squad-details/1
Added player: MS Dhoni
Fetching player details from: https://www.iplt20.com/teams/chennai-super-kings/squad-details/44
Added player: Parthiv Patel
Fetching player details from: https://www.iplt20.com/teams/chennai-super-kings/squad-details/2
Added player: Abhinav Mukund
Fetching player details from: https://www.iplt20.com/teams/chennai-super-kings/squad-details/11
Added player: Srikkanth Anirudha
Fetching player details from: https://www.iplt20.com/teams/chennai-super-kings/squad-details/12
Added player: Subramaniam Badrinath
Fetching player details from: https://www.iplt20.com/teams/chennai-super-kings/squad-details/14
Added player: Suresh Raina
Fetching player details from: https://www.iplt20.com/teams/chennai-super-kings/squad-details/20
Added player: Michael Hussey
Fetchi

In [1]:
df = pd.read_csv("ipl_player_details.csv")

NameError: name 'pd' is not defined

In [8]:
df.dropna(inplace=True)

In [12]:
df['Image URL'][0]

'https://documents.iplt20.com/ipl/IPLHeadshot2024/57.png'

In [13]:
df.duplicated().sum()

1724

In [15]:
df.drop_duplicates(inplace=True)

In [17]:
df.to_csv("ipl_players_cleaned.csv",index=False)